# DeepSearch run-to-run consistency and GO coverage

This notebook summarizes reproducibility between paired Perplexity DeepSearch runs and how well the inferred gene programs align with GO enrichment results.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

DATA_DIR = Path('data')
ANALYSIS_DIR = Path('analysis')
FIG_DIR = ANALYSIS_DIR / 'figures'
FIG_DIR.mkdir(parents=True, exist_ok=True)

matches = pd.read_csv(DATA_DIR / 'deepsearch_program_matches.csv')
runs = pd.read_csv(DATA_DIR / 'deepsearch_runs.csv')
unmatched = pd.read_csv(DATA_DIR / 'deepsearch_unmatched_programs.csv')
duplicates = pd.read_csv(DATA_DIR / 'deepsearch_duplicate_runs.csv')
comparison_summary = pd.read_csv(DATA_DIR / 'comparison_summary.csv')
novel_programs = pd.read_csv(DATA_DIR / 'comparison_novel_programs.csv') if (DATA_DIR / 'comparison_novel_programs.csv').exists() else pd.DataFrame()

dup_annotations = set(duplicates[duplicates['duplicate']]['annotation'])
print(f"Duplicate annotations flagged: {sorted(dup_annotations)}")


## Run-to-run consistency

Metrics are computed per annotation after removing duplicated runs (identical outputs).

In [ ]:
# compute per-annotation stats
records = []
valid_matches = matches[~matches['annotation'].isin(dup_annotations)]
valid_runs = runs[~runs['annotation'].isin(dup_annotations)]
valid_unmatched = unmatched[~unmatched['annotation'].isin(dup_annotations)]
for ann in sorted(valid_runs['annotation'].unique()):
    rc = valid_runs[valid_runs.annotation == ann].groupby('run_index')['program_count'].sum()
    grp = valid_matches[valid_matches.annotation == ann]
    unmatched_count = (valid_unmatched['annotation'] == ann).sum()
    records.append({
        'annotation': ann,
        'run1_programs': int(rc.get(1, 0)),
        'run2_programs': int(rc.get(2, 0)),
        'matched_pairs': len(grp),
        'min_programs': int(rc.min()) if not rc.empty else 0,
        'match_rate': len(grp) / rc.min() if not rc.empty else np.nan,
        'mean_gene_jaccard': grp['gene_jaccard'].mean() if not grp.empty else np.nan,
        'mean_name_similarity': grp['name_similarity'].mean() if not grp.empty else np.nan,
        'mean_combined_similarity': grp['combined_similarity'].mean() if not grp.empty else np.nan,
        'unmatched_programs': unmatched_count,
    })
run_stats = pd.DataFrame(records).sort_values('annotation').reset_index(drop=True)
run_stats


In [ ]:
overall_mean = valid_matches['combined_similarity'].mean()
overall_median = valid_matches['combined_similarity'].median()
mean_match_rate = run_stats['match_rate'].mean()
print(f"Overall combined similarity: mean={overall_mean:.3f}, median={overall_median:.3f}")
print(f"Average match coverage (matched pairs / smaller run): {mean_match_rate:.2f}")
if not run_stats.empty:
    best_row = run_stats.loc[run_stats['mean_combined_similarity'].idxmax()]
    worst_row = run_stats.loc[run_stats['mean_combined_similarity'].idxmin()]
    print(f"Best combined similarity: {best_row['annotation']} ({best_row['mean_combined_similarity']:.3f})")
    print(f"Lowest combined similarity: {worst_row['annotation']} ({worst_row['mean_combined_similarity']:.3f})")
print(f"Total unmatched programs (excluding duplicates): {len(valid_unmatched)}")


In [ ]:
# plot match rates and mean similarities
plot_order = run_stats.sort_values('mean_combined_similarity', ascending=False)['annotation']
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].bar(plot_order, run_stats.set_index('annotation').loc[plot_order, 'match_rate'], color='#4C72B0')
axes[0].set_ylabel('Match coverage (matched pairs / smaller run)')
axes[0].set_xlabel('Annotation')
axes[0].set_ylim(0, 1.1)
axes[0].tick_params(axis='x', rotation=60)
axes[0].set_title('Run-to-run program pairing')

axes[1].bar(plot_order, run_stats.set_index('annotation').loc[plot_order, 'mean_combined_similarity'], color='#55A868')
axes[1].set_ylabel('Mean combined similarity')
axes[1].set_xlabel('Annotation')
axes[1].set_ylim(0, 1.05)
axes[1].tick_params(axis='x', rotation=60)
axes[1].set_title('Similarity of matched programs')

fig.tight_layout()
fig.savefig(FIG_DIR / 'consistency_metrics.png', dpi=200, bbox_inches='tight')
fig


In [ ]:
# distribution of combined similarities per annotation
ordered_ann = plot_order.tolist()
subset = valid_matches.copy()
subset['annotation'] = pd.Categorical(subset['annotation'], categories=ordered_ann, ordered=True)
fig, ax = plt.subplots(figsize=(12, 5))
box_data = [subset[subset['annotation'] == ann]['combined_similarity'] for ann in ordered_ann]
ax.boxplot(box_data, labels=ordered_ann, showmeans=True, meanline=True, patch_artist=True,
           boxprops=dict(facecolor='#CFE8F3', color='#4C72B0'),
           medianprops=dict(color='#C44E52'),
           meanprops=dict(color='#55A868'))
ax.set_ylabel('Combined similarity')
ax.set_xlabel('Annotation')
ax.set_ylim(0, 1.05)
ax.tick_params(axis='x', rotation=60)
ax.set_title('Distribution of combined similarity per annotation')
fig.tight_layout()
fig.savefig(FIG_DIR / 'combined_similarity_box.png', dpi=200, bbox_inches='tight')
fig


## GO enrichment coverage

Coverage is calculated from the comparison markdown files; sets without a parsed GO table are excluded.

In [ ]:
go_summary = comparison_summary.copy()
go_summary['go_match_pct'] = go_summary['matched_go_terms_estimated'] / go_summary['total_gsea_terms'] * 100
missing_annotations = sorted(set(run_stats['annotation']) - set(go_summary['annotation']))
print('Gene sets missing GO comparison tables:', missing_annotations)
go_summary[['annotation', 'total_gsea_terms', 'matched_go_terms_estimated', 'unmatched_go_terms_reported', 'go_match_pct']].sort_values('annotation')


In [ ]:
total_terms = go_summary['total_gsea_terms'].sum()
matched_terms = go_summary['matched_go_terms_estimated'].sum()
unmatched_terms = go_summary['unmatched_go_terms_reported'].sum()
print(f"Aggregate GO coverage: {matched_terms}/{total_terms} terms matched ({matched_terms/total_terms*100:.1f}%)")
if not go_summary.empty:
    min_row = go_summary.loc[go_summary['go_match_pct'].idxmin()]
    print(f"Lowest coverage: {min_row['annotation']} ({min_row['go_match_pct']:.1f}% of {int(min_row['total_gsea_terms'])} terms)")


In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
sorted_go = go_summary.sort_values('go_match_pct')
ax.bar(sorted_go['annotation'], sorted_go['go_match_pct'], color='#8172B2')
ax.set_ylabel('% GO terms matched')
ax.set_xlabel('Annotation')
ax.set_ylim(0, 110)
ax.set_title('GO term coverage by gene set')
ax.tick_params(axis='x', rotation=60)
fig.tight_layout()
fig.savefig(FIG_DIR / 'go_coverage_bar.png', dpi=200, bbox_inches='tight')
fig


## Novel DeepSearch programs (no GO match)

Programs listed as unmatched in the comparison files are ranked by supporting gene count.

In [ ]:
if novel_programs.empty:
    print('No unmatched programs detected in comparison files.')
else:
    display_cols = ['annotation', 'program_name', 'supporting_gene_count']
    top_novel = novel_programs.sort_values(['supporting_gene_count', 'program_name'], ascending=[False, True]).reset_index(drop=True)
    top_novel.head(15)[display_cols]


## Limitations

- Only two DeepSearch runs per gene set; variance estimates are not robust.
- Two metamodels produced duplicate runs, which were excluded from similarity summaries.
- GO coverage relies on the available comparison markdowns; missing or malformed tables leave gaps.
- Similarity metric is heuristic (50% gene Jaccard + 50% name/embedding overlap); orthogonalization of embeddings is not yet applied.